In [1]:
import yaml
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%config InlineBackend.figure_format = 'retina'
import numpy as np

import sys
sys.path.insert(0, "../../scripts/")
from utils import flatten_list

from pymatgen.io.vasp import Xdatcar
from scipy.stats import gaussian_kde
from pytim.gaussian_kde_pbc import gaussian_kde_pbc # magic!

/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
with open('../md_runs.yaml', 'r') as f:
    md_runs = yaml.safe_load(f)
print(md_runs)

{'Li6PS5I': {'0p': [1, 2, 3, 4, 5, 6], '50p': [1, 2, 3, 4, 5, 6, 7], '100p': [1, 2, 3, 4, 5, 6, 7]}, 'Li6PS5Cl': {'0p': [1, 2, 3, 4, 5], '50p': [1, 2, 3, 4, 5, 6], '100p': [1, 2, 3, 4, 5, 6]}}


In [3]:
data_dir = '../../data'
xd = Xdatcar(f'{data_dir}/Li6PS5I/0p/run1/actual_XDATCAR.gz')
structures = xd.structures

In [16]:
coords_list = []
atom_indices = [ i for i, s in enumerate(structures[0]) if s.species_string is 'Li' ]
gaussian_width = 0.035
for this_struc in structures:
    structure = this_struc
    cell_lengths = this_struc.lattice.abc
    coords_list.append(structure.cart_coords[atom_indices])
cart_coords = np.vstack(coords_list)
print(cart_coords.shape)
kde = gaussian_kde_pbc(cart_coords.T, bw_method=gaussian_width)


(26880, 3)


In [17]:
nbins = [120,120,120]
density = np.zeros(nbins)
for i, x in enumerate(tqdm_notebook(np.linspace(0, cell_lengths[0], num=nbins[0]))):
    for j, y in enumerate(tqdm_notebook(np.linspace(0, cell_lengths[0], num=nbins[0]), leave=False)):
        for k, z in enumerate(np.linspace(0, cell_lengths[0], num=nbins[0])):
            density[i,j,k] = kde([x,y,z])

In [15]:
filename = 'test_density.vasp'
structures[0].to(fmt='poscar', filename=filename)

count = 1
with open(filename, 'a') as f:
    f.write('\n')
    f.write(f'{nbins[0]} {nbins[1]} {nbins[2]}\n')
    for i in range(nbins[0]):
        for j in range(nbins[1]):
            for k in range(nbins[2]):
                f.write(" {0:.10e} ".format(density[k,j,i]))
                if count % 5 == 0:
                    f.write("\n")
                count += 1

    

In [ ]:
from scipy.spatial import cKDTree

class gaussian_kde_pbc(gaussian_kde):
    # note that here "points" are those on the grid

    def evaluate_pbc_fast(self, points):
        grid = points
        pos = self.pos
        box = self.box
        d = self.sigma * 2.5
        results = np.zeros(grid.shape[1], dtype=float)
        gridT = grid[::-1].T[:]
        tree = cKDTree(gridT, boxsize=box)
        # the indices of grid elements within distane d from each of the pos
        scale = 2. * self.sigma**2
        indlist = tree.query_ball_point(pos, d)
        for n, ind in enumerate(indlist):
            dr = gridT[ind, :] - pos[n]
            cond = np.where(dr > box / 2.)
            dr[cond] -= box[cond[1]]
            cond = np.where(dr < -box / 2.)
            dr[cond] += box[cond[1]]
            dens = np.exp(-np.sum(dr * dr, axis=1) / scale)
            results[ind] += dens

        return results